<a href="https://colab.research.google.com/github/Alanapengilley/Dampier/blob/main/Dampier_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#load in packages required 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Geological dataset consists of 

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Source_data.csv to Source_data.csv


In [5]:
#load in geological dataset 
import io
source_data = pd.read_csv(io.BytesIO(uploaded['Source_data.csv']))

Elements 

In [6]:
#X = source_data.iloc[:,5:21].values
#y = source_data.iloc[:,3].values

In [33]:
X = source_data[['Al2O3', 'SiO2', 'P2O5', 'K2O', 'CaO', 'TiO2', 'V', 'MnO', 'FeO2O3', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr', 'Y', 'Zr']]
y = source_data['Geochem']

In [27]:
print(X)

     Al2O3   SiO2  P2O5   K2O   CaO  TiO2      V   MnO  FeO2O3     Ni     Cu  \
0    17.58  57.33  0.16  0.51  8.41  0.61  240.0  0.16    3.27   10.0   75.0   
1    15.00  66.28  0.16  0.88  5.15  0.48   97.0  0.14    1.91    0.2   17.0   
2    15.13  66.43  0.14  0.90  5.24  0.47   94.0  0.14    2.74    0.2   15.0   
3    14.74  69.44  0.16  1.03  4.38  0.40   43.0  0.12    1.73    0.2    9.0   
4    14.63  65.80  0.15  0.88  5.31  0.48   80.0  0.15    1.71    0.1    9.0   
..     ...    ...   ...   ...   ...   ...    ...   ...     ...    ...    ...   
309  16.80  52.30  0.70  2.05  7.95  1.48  280.0  0.16    2.45   33.0  100.0   
310  17.10  58.50  0.50  2.70  6.40  1.01  130.0  0.16    3.55    8.0   22.0   
311  14.10  47.10  0.42  0.95  9.90  1.28  280.0  0.18    3.45  222.0   68.0   
312  16.20  52.50  0.50  1.80  8.20  1.01  195.0  0.18    2.95   97.0   52.0   
313  17.30  60.20  0.34  2.50  6.00  0.68  120.0  0.12    2.95   10.0   42.0   

       Zn    Rb     Sr     Y     Zr    

In [51]:
print(y)

0               E
1               E
2               E
3               E
4               E
          ...    
309    New Guinea
310    New Guinea
311    New Guinea
312    New Guinea
313    New Guinea
Name: Geochem, Length: 314, dtype: object


In [10]:
from google.colab import files
uploaded = files.upload()

Saving dampier_data.csv to dampier_data.csv


In [11]:
#load in artefact data 
dampier_data = pd.read_csv(io.BytesIO(uploaded['dampier_data.csv']))

In [12]:
#testX = dampier_data.iloc[:,3:19].values
#print(testX)

[[3.65e+01 0.00e+00 5.00e-01 1.09e+01 6.00e-01 9.88e+01 1.00e-01 9.80e+00
  6.30e+01 7.20e+01 8.70e+01 8.00e+00 2.76e+02 1.90e+01 1.30e+01 6.00e+00]]


In [36]:
dampier_axe = dampier_data[['Al2O3', 'SiO2', 'P2O5', 'K2O', 'CaO', 'TiO2', 'V', 'MnO', 'FeO2O3', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr', 'Y', 'Zr']]
textX = dampier_axe.values.tolist()
print(textX)

[[15.3, 36.5, 0.0, 0.5, 10.9, 0.6, 98.8, 0.1, 9.8, 63.0, 72.0, 87.0, 8.0, 276.0, 19.0, 13.0]]


In [52]:
dampier_axe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Al2O3   1 non-null      float64
 1   SiO2    1 non-null      float64
 2   P2O5    1 non-null      int64  
 3   K2O     1 non-null      float64
 4   CaO     1 non-null      float64
 5   TiO2    1 non-null      float64
 6   V       1 non-null      float64
 7   MnO     1 non-null      float64
 8   FeO2O3  1 non-null      float64
 9   Ni      1 non-null      int64  
 10  Cu      1 non-null      int64  
 11  Zn      1 non-null      int64  
 12  Rb      1 non-null      int64  
 13  Sr      1 non-null      int64  
 14  Y       1 non-null      int64  
 15  Zr      1 non-null      int64  
dtypes: float64(8), int64(8)
memory usage: 256.0 bytes


In [53]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaler = sc.fit(X)
trainX_scaled = scaler.transform(X)
testX_scaled = scaler.transform(testX)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [73]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 3)
model = lda.fit(trainX_scaled, y)
print(model)

LinearDiscriminantAnalysis(n_components=3)


In [68]:
lda.explained_variance_ratio_.sum()

0.8801229176051104

In [74]:
trainX_lda = lda.transform(trainX_scaled)
testX_lda =lda.transform(testX_scaled)

In [ ]:
print(trainX_lda)

In [76]:
print(testX_lda)

[[262.49041622  39.68199101 534.7439979 ]]


In [77]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

#Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#evaluate model
scores = cross_val_score(model, trainX_scaled, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores))   
print(scores)

0.8599462365591397
[0.9375     0.71875    0.875      0.8125     0.80645161 0.90322581
 0.90322581 0.90322581 0.87096774 0.90322581 0.90625    0.875
 0.9375     0.90625    0.77419355 0.77419355 0.74193548 0.87096774
 0.90322581 0.87096774 0.875      0.71875    0.75       0.9375
 0.90322581 0.87096774 0.90322581 0.87096774 0.83870968 0.93548387]


In [108]:
#from sklearn.linear_model import LogisticRegression
#logReg = LogisticRegression()
#logReg.fit(trainX_lda, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

predict() can be executed on a trained model in order to predict the actual label over a new set of data. 

In [91]:
from sklearn.metrics import accuracy_score
pred_test = model.predict(testX_scaled)
#print('Accuracy: {:.2f}'.format(accuracy_score(y, y_pred)))



predict_proba() returns the class probabilities for each data point. 

In [103]:
y_pred_test = model.predict_proba(testX_scaled)
classes = model.classes_
print(y_pred_test)

#for class_name, proba in zip(classes, y_pred_test):
    #print(f"{class_name}: {proba}")


[[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  7.37269358e-113 1.00000000e+000 3.36213046e-040 0.00000000e+000]]


In [49]:
print(classes)

['E' 'F' 'GN' 'GS' 'H' 'New Guinea' 'Rabaul' 'Vitiaz']


In [97]:
np.max(model.predict_proba(testX_scaled))

1.0

In [93]:
ix = y_pred_test.argmax(1).item()
list = []
list.append(pred_test[ix])
list.append(f'{y_pred_test[0,ix]:.2%}')


def get_predict_proba(row, model):
return max(model.predict_proba(row.values.reshape(1,-1)))

IndexError: ignored

In [111]:
print(model.get_params())

{'covariance_estimator': None, 'n_components': 3, 'priors': None, 'shrinkage': None, 'solver': 'svd', 'store_covariance': False, 'tol': 0.0001}


In [83]:
#y_pred_train = logReg.predict(trainX_lda)
y_pred = model.predict(trainX_scaled)
y_pred_train = model.predict_proba(trainX_scaled)


In [84]:
highest_prob = []

for sample in range(y_pred_train.shape[0]):    #iterates through each sample
    highest_prob.append(np.max(y_pred_train[sample]))   #gets max probability of that sample (highest likelyhood)
    
    
print(len(highest_prob))

314


Geochem regions 

In [85]:
df = pd.DataFrame({'sample':source_data.Name.values, 'predicted_cat':y_pred, 'prob_predicted':highest_prob, 'actual_cat':source_data.Geochem})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(df)

               sample predicted_cat  prob_predicted  actual_cat
0         E1:68490025             F        0.848423           E
1         E1:68490031             F        0.822661           E
2         E1:68490033             F        0.902943           E
3         E1:68490030             F        0.784486           E
4         E1:75710021             F        0.740494           E
5          E1:WP01001            GN        0.865929           E
6           E1:W00013            GN        0.894250           E
7          E1:WP01003            GN        0.935488           E
8          E1:WP01002            GN        0.964281           E
9         E2:51NG3028             E        0.984260           E
10        E2:51NG3035             E        0.909428           E
11       E2:51NG3039B             E        0.885431           E
12        E3:51NG0113             E        0.749563           E
13        E3:51NG0199             E        0.892404           E
14        E3:51NG0202             E     

In [86]:
print('classes: ', lda.classes_)
print('Priors: ', lda.priors_)
print('Explained variance ratio: ', lda.explained_variance_ratio_)

classes:  ['E' 'F' 'GN' 'GS' 'H' 'New Guinea' 'Rabaul' 'Vitiaz']
Priors:  [0.27388535 0.20063694 0.10509554 0.06687898 0.08280255 0.05095541
 0.17197452 0.0477707 ]
Explained variance ratio:  [0.49708357 0.23986742 0.14317192]


In [87]:
import plotly.express as px
fig = px.scatter(source_data, x=trainX_lda[:,0], y=trainX_lda[:,1],
                 labels = {
                     "x": "LDA 1",
                     "y": 'LDA 2',
                 },
                 opacity=1, color=y)

# Set figure title
fig.update_layout(title_text="LDA scatter plot of training data")

# Update marker size
fig.update_traces(marker=dict(size=4))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.show()

In [122]:
import plotly.express as px
fig = px.scatter(dampier_data, x=testX_lda[:,0], y=testX_lda[:,1],
                 labels = {
                     "x": "LDA 1",
                     "y": 'LDA 2',
                 },
                 opacity=1)

# Set figure title
fig.update_layout(title_text="LDA scatter plot of training data")

# Update marker size
fig.update_traces(marker=dict(size=4))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.show()